# Interactive Map - Folium (Leaflet)

For experimentation on creating an interactive map with our data.

In [1]:
### Import Libraries

# File manipulation

import os # For working with Operating System
from sys import platform # Diagnose operating system
import json # For working with Json files
import requests # Processing online requests


# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import geopandas as gpd # Spatial Data Manipulation

# Visualization

import folium # Interactive Leaflet.js mapping 
from folium import features

import warnings
warnings.filterwarnings('ignore') # Ignores some warnings

In [2]:
### Definitions

cwd = os.getcwd() # Current Working Directory

# Forward or back slashs for filepaths? <- Not sure here. Only know Windows & Linux

if platform == "linux" or platform == "linux2":
    slash = '/'
elif platform == 'win32':
    slash = '\\'

# Load Data

In [4]:
# Load Data

# Get GeoJsons from GitHub

# Base URL

url = (
    "https://raw.githubusercontent.com/RwHendrickson/MappingGZ/main/Prototype/Notebooks/CleaningData"
)

# Minneapolis Boundary

mpls_boundary_path = f"{url}/Boundary/mpls_boundaryWGS84.geojson"

# GZ Boundaries

gz_boundary_path = f"{url}/GZ_Boundaries/gz_boundaryWGS84.geojson"

# Get bigger files from computer

cwd = os.getcwd()

mpls_aadt_path = cwd + slash + 'CleaningData' + slash + 'Traffic' + slash + 'mpls_aadtWGS84.geojson'
mpls_emissions_path = cwd + slash + 'CleaningData' + slash + 'PermittedEmissions' + slash + 'mpls_emissions.csv'
mpls_health_path = cwd + slash + 'CleaningData' + slash + 'Health' + slash + 'mpls_health_tracts' + slash + 'mpls_health_tracts.shp'
mpls_census_path = cwd + slash + 'CleaningData' + slash + 'Demographics' + slash + 'mpls_census_data' + slash + 'mpls_census_data.shp'
mpls_WIMN_path = cwd + slash + 'CleaningData' + slash + 'WhatsInMyNeighborhood' + slash + 'minneapolis_my_neighborhood_sites.shp'

# Load files as geodataframes for focusing data

traffic = gpd.read_file(mpls_aadt_path)
emissions = pd.read_csv(mpls_emissions_path)
health = gpd.read_file(mpls_health_path)
demos = gpd.read_file(mpls_census_path)
wimn = gpd.read_file(mpls_WIMN_path)

emissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))


"\ntraffic = gpd.read_file(mpls_aadt_path)\nemissions = pd.read_csv(mpls_emissions_path)\nhealth = gpd.read_file(mpls_health_path)\ndemos = gpd.read_file(mpls_census_path)\nwimn = gpd.read_file(mpls_WIMN_path)\n\nemissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))\n"

In [8]:
# Focus Data

## Traffic

aadt = traffic[['SEQUENCE_N', 'ROUTE_LABE', 'CURRENT_VO', 'geometry']]
big_aadt = aadt[aadt['CURRENT_VO'] > 10000]

# Emissions

emissions_2020 = emissions_geo[emissions_geo['YEAR']==2020][['FACILITY_NAME', 'INDUSTRY_TYPE', 'NAICS_CODE', 'POLLUTANT', 'LBS_EMITTED','geometry']]
big_vocs = emissions_2020[(emissions_2020['POLLUTANT'] == 'Volatile Organic Compounds') & 
                         (emissions_2020['LBS_EMITTED'] > 1000)]
big_pm = emissions_2020[(emissions_2020['POLLUTANT'] == 'PM2.5 Primary') & 
                         (emissions_2020['LBS_EMITTED'] > 1000)]

# Focusing Health Data
health_focus = health[['GEOID10', 'ACCESS2_Cr', 'CANCER_Cru', 'CASTHMA_Cr', 'DEPRESSION', 'OBESITY_Cr', 'geometry']]

# Focusing Census Data
demo_focus = demo[['Income', 'Total_Coun', 'White_Coun', 'GEOID10', 'geometry']]


# What's In My Neighborhood?
active_my_nabe_df = wimn[wimn['active_fla'] == 'Y'][['name', 'active_fla', 'address_st', 'activity', 'program_na', 'program__1', 'mpca_id_li','industrial', 'shape', 'geometry']]
# Convert to EPSG:4326 for mapping
active_my_nabe_df = active_my_nabe_df.to_crs('EPSG:4326')


"\n# What's In My Neighborhood?\nactive_my_nabe_df = wimn[wimn['active_fla'] == 'Y'][['name', 'active_fla', 'address_st', 'activity', 'program_na', 'program__1', 'mpca_id_li','industrial', 'shape', 'geometry']]\n# Convert to EPSG:4326 for mapping\nactive_my_nabe_df = active_my_nabe_df.to_crs('EPSG:4326')\n"

# Define Visualizations

## What's in My Neighborgood

In [ ]:
#Format into seperate layers by program name for clarity in mapping the big dataset

#create list of unique program_na values
unique_program_nas = active_my_nabe_df['program_na'].drop_duplicates()

# define a function to  sort the dataframe into smaller data frames based on the unique values of the program_na
def sort_dfs(unique_program_na):
    program_type_df = active_my_nabe_df[active_my_nabe_df['program_na'] == unique_program_na]
    return program_type_df

#create an empty list to store dataframes
list_of_unique_program_na_dfs = []  

#loop through unique doc nums and add the sorted dataframe to the above list on every iteration:
for unique_program_na in unique_program_nas:
    list_of_unique_program_na_dfs.append(sort_dfs(unique_program_na))

unique_program_nas.values # Program names -set as variable for use later


### Styling WIMN data - colors by layer

In [ ]:
#Create Color dictionary for hexidecimal colors, key is unique values in the selected data set, values are hexidecimal colors for nominal data

color_dict = {'Hazardous Waste':'#ff4da6',
              'Investigation and Cleanup':'#ffcc00',
              'Stormwater':'#660033',
              'Multiple Programs':'#80ffff',
              'Tanks':'#4d94ff',
              'SSTS':'#8533ff',
              'Air Quality':'#1a1aff',
              'Solid Waste':'#cc6600',
              'Water Quality':'#00995c',
              'Environmental Review':'#ff9900'}

### Popups for WIMN data

In [ ]:
# Add Popups

column_names = list_of_unique_program_na_dfs[0].columns # Column names
print(column_names)

In [ ]:
def popup_html(point):
    '''Creates a popup with basic info'''
    
    # Color for the table
    
    left_col_color = "#627c6a"
    right_col_color = "#bccdb9"
    
    # Fields to present
    
    name = point['name'] + '<br/>' + str(point['address_st']) # this is parsed as HTML, so use HTML style for line break
    activity = point['activity']
    industrial = point['industrial']
    
    # Add it as an html script!? - Template: https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(name) + """

</head>
    <table style="height: 100px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Industrial Classification</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(industrial) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Activity:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(activity) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

## Annual Average Daily Traffic

In [ ]:
### AADT style

def style_aadt(feature):
    '''Set Thickness/color of streets to be proportional to traffic volume'''
    
    volume = feature['properties']['CURRENT_VO']
    
    if volume < 1773: # Low volume
        return {
        "weight": 0.5,
        "color": "#848484"
    }
    elif volume < 5378: # Low-mid volume
        return {
        "weight": 1,
        "color": "#936d6d"
    }
    elif volume < 16308: # Mid volume
        return {
        "weight": 2,
        "color": "#a94646"
    }
    elif volume < 100000: # High Volume
        return {
        "weight": 5,
        "color": "#f90707"
    }
    else: # Very High Volume Potentially could use another break here
        return {
        "weight": 10,
        "color": "#090707"
    }
    

## Permitted Emissions

### Popups

In [ ]:
def emissions_popup_html(point):
    '''Creates a popup with basic info on an industrial polluter'''

    left_col_color = "#627c6a"
    right_col_color = "#bccdb9"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(point.FACILITY_NAME) + """

</head>
    <table style="height: 100px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Pollutant:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.POLLUTANT) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Pounds Emitted (2020):</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.LBS_EMITTED) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Industry Type:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.INDUSTRY_TYPE) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">NAICS code:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.NAICS_CODE) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

### VOC Emissions

In [ ]:
# Plot the facilities' VOC emission
    
def plot_voc(point):
    '''plot circle of facilities related to amount of vocs emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    popup = emissions_popup_html(point)
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       fill = True,
                       popup = popup
                       ).add_to(voc_feature_group)

## PM2.5 Emissions

In [ ]:
# Plot the facilities' VOC emission
    
def plot_pm(point):
    '''plot circle of facilities related to amount of particulate matter 2.5 emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    popup = emissions_popup_html(point)
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       fill = True,
                       popup = popup
                       ).add_to(pm_feature_group)

## Health and Census Data

In [ ]:
# Merge Health & Census Data

demo_focus['GEOID10'] = demo_focus['GEOID10'].astype(str) #changing the GEOID10 column to a string
health_focus['GEOID10'] = health_focus['GEOID10'].astype(str)

# Merge tracts and health, remove final two columns from health, renaming geometry_x to geometry

health_demo = pd.merge(health_focus, demo_focus,
                         left_on = 'GEOID10', right_on = 'GEOID10',
                         how = 'inner').iloc[:,0:108].rename(columns = {'geometry_x':'geometry'})
health_demo_full = health_demo.drop(columns=['geometry_y'])
health_demo_full.head(2)

In [ ]:
# Calculating area, population density, and percentages

health_demo_full['White_Per'] = ((health_demo_full['White_Coun']/health_demo_full['Total_Coun'])*100).fillna(0).astype(int)
health_demo_full['NonWhite_Per'] = (100 - (health_demo_full['White_Per'])).fillna(0).astype(int)

health_demo_area = health_demo_full.assign(area = (health_demo_full.area)/(1000**2))
health_demo_area['Pop_Dens'] = (health_demo_area['Total_Coun']/health_demo_area['area']).fillna(0).astype(int)
health_demo_area.sample(10)

In [9]:
# Setting style for popups
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# Population Density
popdens_feature_group = folium.FeatureGroup() # Create Feature Group
a = folium.Choropleth(health_demo_area, 
                  data = health_demo_area, # Get Data
                  key_on = 'feature.properties.GEOID10', # Select key
                  columns = ['GEOID10','Pop_Dens'], # Select column
                  fill_color = 'YlOrBr', # Choose colorscheme
                  legend_name = 'Population Density', # Name legend
                  overlay = True,
                  bins=7,
                  name = 'Population Density', # Name layer
                  show = False
                  )
# Removing legends
for key in a._children:
    if key.startswith('color_map'):
        del(a._children[key])
        

hover_a = folium.features.GeoJson(
    data = health_demo_area,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    popup=folium.features.GeoJsonPopup(
        fields=['Pop_Dens',
                'ACCESS2_Cr',
                'CANCER_Cru',
                'CASTHMA_Cr', 
                'DEPRESSION', 
                'OBESITY_Cr'
               ],
        aliases=['Population Density',
                 'No Access to Healthcare (%)', 
                 'Cancer (%)', 
                 'Asthma (%)', 
                 'Depression (%)', 
                 'Obesity (%)'                 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
a.add_child(hover_a)


# Race: White
white_feature_group = folium.FeatureGroup(name = 'Race: White',
                                           show = False) # Create Feature Group

b = folium.Choropleth(health_demo_area, 
                  data = health_demo_area, # Get Data
                  key_on = 'feature.properties.GEOID10', # Select key
                  columns = ['GEOID10','White_Per'], # Select column
                  fill_color = 'YlOrBr', # Choose colorscheme
                  legend_name = 'Race: White (%)', # Name legend
                  overlay = True,
                  bins=7,
                  name = 'Race: White', # Name layer
                  show = False
                  )
# Removing legends
for key in b._children:
    if key.startswith('color_map'):
        del(b._children[key])
        
hover_b = folium.features.GeoJson(
    data = health_demo_area,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    popup=folium.features.GeoJsonPopup(
        fields=['White_Per',
                'ACCESS2_Cr',
                'CANCER_Cru',
                'CASTHMA_Cr', 
                'DEPRESSION', 
                'OBESITY_Cr'
               ],
        aliases=['Race: White (%)',
                 'No Access to Healthcare (%)', 
                 'Cancer (%)', 
                 'Asthma (%)', 
                 'Depression (%)', 
                 'Obesity (%)'                 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
b.add_child(hover_b)


# Race: Non-White
nonwhite_feature_group = folium.FeatureGroup(name = 'Race: Non-White',
                                           show = False) # Create Feature Group

c = folium.Choropleth(health_demo_area, 
                  data = health_demo_area, # Get Data
                  key_on = 'feature.properties.GEOID10', # Select key
                  columns = ['GEOID10','NonWhite_Per'], # Select column
                  fill_color = 'YlOrBr', # Choose colorscheme
                  legend_name = 'Race: Non-White (%)', # Name legend
                  overlay = True,
                  bins=7,
                  name = 'Race: Non-White', # Name layer
                  show = False
                  )
# Removing legends
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])

hover_c = folium.features.GeoJson(
    data = health_demo_area,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    popup=folium.features.GeoJsonPopup(
        fields=['NonWhite_Per',
                'ACCESS2_Cr',
                'CANCER_Cru',
                'CASTHMA_Cr', 
                'DEPRESSION', 
                'OBESITY_Cr'
               ],
        aliases=['Race: Non-White (%)',
                 'No Access to Healthcare (%)', 
                 'Cancer (%)', 
                 'Asthma (%)', 
                 'Depression (%)', 
                 'Obesity (%)'                 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
c.add_child(hover_c)

# Income
income_feature_group = folium.FeatureGroup(name = 'Average Income',
                                           show = False) # Create Feature Group

d = folium.Choropleth(health_demo_area, 
                  data = health_demo_area, # Get Data
                  key_on = 'feature.properties.GEOID10', # Select key
                  columns = ['GEOID10','Income'], # Select column
                  fill_color = 'YlOrBr', # Choose colorscheme
                  legend_name = 'Average Income ($)', # Name legend
                  overlay = True,
                  bins=7,
                  name = 'Average Income', # Name layer
                  show = False
                  )
# Removing legends
for key in d._children:
    if key.startswith('color_map'):
        del(d._children[key])

hover_d = folium.features.GeoJson(
    data = health_demo_area,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    popup=folium.features.GeoJsonPopup(
        fields=['Income',
                'ACCESS2_Cr',
                'CANCER_Cru',
                'CASTHMA_Cr', 
                'DEPRESSION', 
                'OBESITY_Cr'
               ],
        aliases=['Average Income',
                 'No Access to Healthcare (%)', 
                 'Cancer (%)', 
                 'Asthma (%)', 
                 'Depression (%)', 
                 'Obesity (%)'                 
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
d.add_child(hover_d)

NameError: name 'health_demo_area' is not defined

# Add to the map!

In [ ]:
# Make a basic folium map

m = folium.Map(location=[44.986656, -93.258133],
               zoom_start=12,
               tiles = 'cartodbpositron')

# Add Minneapolis boundary

folium.GeoJson(mpls_boundary_path,
               style_function = lambda feature: {'color':'#000000', # Border color
                                                'fillOpacity' : '0'}, # Fill transparency
               name="Minneapolis Boundary", # Layer's Name
               control = False # Can you toggle in layer control?
              ).add_to(m)


# Add GZ boundaries

folium.GeoJson(gz_boundary_path, 
               style_function = lambda feature: {'fillColor':'#84E884',
                                 'color':'#003700',
                                 'fillOpacity': '0.5',
                                 'weight': '0'}, # Thickness of lines
               name = "Green Zone Boundaries",
               control = False).add_to(m)

# Add traffic

folium.GeoJson(big_aadt,
               style_function = lambda feature: style_aadt(feature), 
               name = 'Annual Average Daily Traffic',
               show = False # Does this appear in default of map
              ).add_to(m)

# Add Volatile Organic Compounds

voc_feature_group = folium.FeatureGroup(name = 'VOC Emitters',
                                           show = False) # Create Feature Group
big_vocs.apply(plot_voc, axis = 1) # Add to group
m.add_child(voc_feature_group)

# Add PM2.5

pm_feature_group = folium.FeatureGroup(name = 'PM2.5 Emitters',
                                           show = False) # Create Feature Group
big_pm.apply(plot_pm, axis = 1) # Add to group
m.add_child(pm_feature_group)

# Add Census Data

    #Population Density

m.add_child(a)    

    #Race: White

m.add_child(b)

    #Race: Non-White
    
m.add_child(c)

    #Income

m.add_child(d)


# Add WIMN Data

# Get iterables

programs = unique_program_nas.values # Program names
program_datasets = list_of_unique_program_na_dfs # Corresponding Datasets

for data in program_datasets:
    
    program = np.unique(data['program_na'].values)[0] # Program Name(s)    
    layer_name = program + ' - MPCA' # Layer Name
    color = color_dict[program] # Color for the layer

    # Add to map
    
    ## Create Feature Group
    
    feature_group = folium.FeatureGroup(name = layer_name,
                                           show = False)
    
    ## Add all points to layer
    
    for index, point in data.iterrows():
        
        popup = popup_html(point) # Get popup value
        
        # Add point as circle
        
        folium.Circle(location=[point['geometry'].y,point['geometry'].x],
                      weight = 2,
                      color = color,
                      fill = True,
                      popup =  popup
                      ).add_to(feature_group)
        
        
        
    ## Add layer to map
    
    m.add_child(feature_group)
        

# Turn on layer control

m.add_child(folium.map.LayerControl())

In [ ]:
# Save Map

m.save(os.path.join('.', 'MVP.html'))